In [1]:
import networkx as nx
import pandas as pd
import pickle
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import osmnx as ox
import networkx as nx
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap as Basemap

os.environ['KMP_DUPLICATE_LIB_OK']='True'
Project_Path='/Users/nickkarras/PycharmProjects/ParkingViolationPredictionGraph_Git'

# Import Basic Infos

In [2]:
Sectors=pd.read_csv(Project_Path+ '/Data/Parking_Slot_Proccesed.csv',sep=',', index_col=0)
Nodelist=Sectors[['ΚΩΔΙΚΟΣ ΤΟΜΕΑ','Latitude','Longitude']]
a=Nodelist['ΚΩΔΙΚΟΣ ΤΟΜΕΑ']
Nodelist=Nodelist.drop(['ΚΩΔΙΚΟΣ ΤΟΜΕΑ'], axis=1)
Nodelist.insert(0, "Slot_id", a, True)

Legal_Ilegan=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.3.csv',sep=',',index_col=0)
Unique_Sectors=Legal_Ilegan['Slot_id'].unique()
Unique_Sectors=pd.DataFrame(Unique_Sectors, columns=['Slot_id'])
Nodelist=pd.merge(Unique_Sectors,Nodelist, on='Slot_id')

# Find Duration using API

In [3]:
import json
import openrouteservice

def get_Duration(long1,lat1,long2,lat2):
    coords = ((long1,lat1),(long2,lat2))
    client = openrouteservice.Client(key='test')
    res = client.directions(coords)
    #set location coordinates in longitude,latitude order

    #call API
    res = client.directions(coords)
    #test our response
    with(open(Project_Path+'Data/ETA.json','+w')) as f:
     f.write(json.dumps(res,indent=4, sort_keys=True))


    f = open('ETA.json')

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Iterating through the json
    # list
    duration=data['routes'][0]['segments'][0]['duration']
    #Edge_Weights.append(Time_Distance)

    # Closing file
    f.close()
    return duration

In [13]:
def Normalize_Edgedata(Targets):
    Targets = (((0.9-0.1) * (Targets - np.min(Targets))) / (np.max(Targets) - np.min(Targets))) + 0.1
    return Targets

# Find All Combinations

In [20]:
List=Nodelist.values.tolist()
x=[]
for i in range (0,len(List)):
    for k in range (i,len(List)):
         if List[i][0]!=List[k][0]:
            ed=[]
            ed.append(List[i])
            ed.append(List[k])
            x.append(ed)

Edge_W=[]

for i in tqdm(range (0,len(x))):
    Distance=[]
    b= geopy.distance.geodesic((x[i][1][1],x[i][1][2]), (x[i][0][1],x[i][0][2])).m
    if b<250:
        
        d=get_Duration(x[i][1][2],x[i][1][1], x[i][0][2],x[i][0][1])
        Distance.append(x[i][0][0])
        Distance.append(x[i][1][0])
        Distance.append(d)
        Edge_W.append(Distance)
    

100%|██████████| 24531/24531 [00:01<00:00, 12792.13it/s]


In [22]:
a=pd.DataFrame(Edge_W, columns=['Sector_1','Sector_2','Duration'])
a['Sector_2']=a['Sector_2'].astype(int)
a['Sector_1']=a['Sector_1'].astype(int)
a['Duration']=Normalize_Edgedata(a['Duration'])
a

,Sector_1,Sector_2,Duration
0,10,11,0.240442
1,10,17963,0.669492
2,10,18143,0.413026
3,10,18144,0.478163
4,10,18224,0.457742
...,...,...,...
2803,91,94,0.145864
2804,93,94,0.375292
2805,95,96,0.398214
2806,95,98,0.668416


# Add self-edges

In [20]:
T=[]
List=Nodelist.values.tolist()
for i in range (0,len(List)):
    T.append(List[i][0])
T1=pd.DataFrame(T,columns=['Sector_1'])
T1['Sector_2']=T
T1['Duration']=1

Edges2 = pd.concat([a,T1])

In [21]:
Edges2=Edges2.reset_index()
Edges2=Edges2.drop(['index'], axis=1)
Edges2['Sector_1']=Edges2['Sector_1'].astype(int)
Edges2['Sector_2']=Edges2['Sector_2'].astype(int)
Edges2

,Sector_1,Sector_2,Duration
0,10,11,NaN
1,10,17963,NaN
2,10,18143,NaN
3,10,18144,NaN
4,10,18224,NaN
...,...,...,...
3025,93,93,1.0
3026,94,94,1.0
3027,95,95,1.0
3028,96,96,1.0


# Prepare edges for model

In [22]:
with open(Project_Path+'Data/Dataset_Graph.pkl', 'rb') as inp:
    Train_Dataset = pickle.load(inp)
with open(Project_Path+'Data/Test_Dataset_Graph.pkl', 'rb') as inp:
    Test_Dataset = pickle.load(inp)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/nickkarras/PycharmProjects/ParkingViolationPredictionGraph_GitData/Dataset_Graph.pkl'

In [23]:
Edgelist=Edges2
Slot_idx = {name: idx for idx, name in enumerate(sorted(Train_Dataset[6]["Slot_id"].unique()))}
for i in tqdm (range(0,len(Train_Dataset))):
    Train_Dataset[i]["Slot_id"] = Train_Dataset[i]["Slot_id"].apply(lambda name: Slot_idx[name])

for j in tqdm (range(0,len(Test_Dataset))):
    Test_Dataset[j]["Slot_id"] = Test_Dataset[j]["Slot_id"].apply(lambda name: Slot_idx[name])
    
Edgelist["Sector_1"] = Edgelist["Sector_1"].apply(lambda name: Slot_idx[name])
Edgelist["Sector_2"] = Edgelist["Sector_2"].apply(lambda name: Slot_idx[name])

NameError: name 'Train_Dataset' is not defined

In [24]:
Edges2.to_csv(Project_Path+ '/Data/EdgesDuration250.csv')
Edges2

,Sector_1,Sector_2,Duration
0,10,11,NaN
1,10,17963,NaN
2,10,18143,NaN
3,10,18144,NaN
4,10,18224,NaN
...,...,...,...
3025,93,93,1.0
3026,94,94,1.0
3027,95,95,1.0
3028,96,96,1.0
